# Exporting in models
This tutorial provides inspiration and potentially useful code for exporting data in a `pp.Model` based simulation for visualization in e.g. [ParaView](https://www.paraview.org/). While the PorePy [model class](./incompressible_flow_model.ipynb) and the [exporter](./exporter.ipynb) are introduced elsewhere, this tutorial provides some detail on how to combine them. To this end, we will choose the transient model for mixed-dimensional poroelasticity with fracture deformation and adjust it to make sure the solution is exported according to our requirements, i.e. at the right stages of the simulation and that we export the right fields/variables. The tutorial consists of two parts. The first covers standard usage and the second covers more advanced usage related to debugging. 

We start with a very simple case, indicating which methods could be overwritten to specify which variables are exported. In addition to overwriting the \_export method, we introduce a separate method defining the data to be exported when write_vtu is called. We rely on `pp.ContactMechanicsBiot` calling `self._export()` in `after_newton_convergence` and `self.exporter.write_pvd()` in `after_simulation`, which yields an export at each time step and a pvd file giving ParaView access to all time steps.

In [1]:
import porepy as pp
import numpy as np

class Exporting:
    """Mixin class integrating Exporter and a poroelasticity model.
    
    Only methods related to exporting are implemented herein. 
    """
    def _export_data(self):
        """Returns data for exporting.
        
        returns: 
            Any type compatible with data argument of pp.Exporter().write_vtu().
        """
        matrix = [self._nd_subdomain()]
        fractures = self.mdg.subdomains(dim=self.nd-1)
        data =  [
            (matrix, self.displacement_variable),
            self.scalar_variable,  # All subdomains
            (fractures, self.contact_traction_variable),
        ]
        return data
    
    def _export(self):
        """Export current solution to vtu files.
        
        This method is typically called by _after_newton_convergence.
        """
        self.exporter.write_vtu(self._export_data(), time_dependent=True)
        
class ExportingPoroelasticity(Exporting, pp.ContactMechanicsBiot):
    """Combine exporter class and model using multiple inheritance.
    
    Note on inheritance ordering:
    We want to override the model methods with those
    implemented in Exporting, which thus should come first. Methods not 
    implemented in the first parent will be called from the second parent
    class, i.e. very limited risk of messing things up as long as the
    former class is as simple as here.
    """
    pass

        
params = {"folder_name": "model_exporting", "use_ad": True}
model_0 = ExportingPoroelasticity(params)
pp.run_time_dependent_model(model_0, params)

Since the default `create_grid` method of `ContactMechanicsBiot` (inherited from `AbstractModel`) produces a monodimensional domain, we get data files containing pressure and displacement in the matrix domain and no tractions (no fractures are present).
## Mixed-dimensional simulations
We now extend to the mixed-dimensional case. We can use `Exporting` as above. In the parameters, we adjust the file name to avoid overwriting the previous files. If you inspect the suffixes of the files created, you can see how the exporter deals with multiple time steps by default.

In [2]:
class MDPoroelasticity(pp.ContactMechanicsBiot):
    """Modify from single- to mixed-dimensional poroelasticity."""
    def create_grid(self):
        """Create md grid and define the box-shaped domain.
        
        The implemented method of parent class yields a single-dimensional grid.
        Overwrite with a mixed-dimensional one.
        """
        self.mdg, self.box = pp.md_grids_2d.two_intersecting(
            self.params.get("mesh_args", None),
            simplex=False,
        )
        pp.contact_conditions.set_projections(self.mdg)
        
class ExportingMDPoroelasticity(Exporting, MDPoroelasticity):
    pass

params.update({
    "end_time": 2,
    "file_name": "md",
    }
)
model_1 = ExportingMDPoroelasticity(params)
pp.run_time_dependent_model(model_1, params)

C:\Users\Ivar\anaconda3\envs\condapp\lib\site-packages\scipy\sparse\_index.py:137: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray_sparse(i, j, x)


### Tailored exporting
Suppose we want to perform a simulation similar to above, but require more data for visualization. For instance, we might very reasonably want to look at the displacement jump on the fracture. This is not a primary variable, and thus not immediately available in the `pp.STATE` subdictionary of the fracture subdomain dictionary. We implement this as a second mixin class which we combine with `MDPoroelasticity`, facilitating modular reuse of the tailored exporting class in combination with e.g. pp.

In [3]:
class TailoredExporting(Exporting):
    def _export_data(self):
        """Returns data for exporting.
        
        returns: 
            Any type compatible with data argument of pp.Exporter().write_vtu().
        """
        jumps = list()
        fractures = self.mdg.subdomains(dim=self.nd-1)
        vals = self._displacement_jump(fractures).evaluate(self.dof_manager).val
        # The jump values are defined on all fractures. We need to access the values
        # corresponding to individual subdomains, which corresponds to a restriction:
        # FIXME: EK, should we rather use the dof manager for this?
        projection = pp.ad.SubdomainProjections(
            subdomains=fractures, nd=self.nd
        )
        for sd in fractures:
            restriction = projection.cell_restriction([sd]).parse(self.mdg)
            jumps.append((sd, "displacement_jump", restriction * vals))
        data = super()._export_data() + jumps
        return data
    
    def _export(self):
        """Export current solution to vtu files.
        
        This method is typically called by _after_newton_convergence.
        """
        self.exporter.write_vtu(self._export_data(), time_dependent=True)
        
class Tailored(TailoredExporting, MDPoroelasticity):
    pass
    
params.update({
    "end_time": 2,
    "file_name": "jumps",
    }
)
model_2 = Tailored(params)
pp.run_time_dependent_model(model_2, params)

# Iteration exporting for debugging
We now turn to exporting data for each iteration when solving the nonlinear system. This second part is significantly more advanced than the preceeding part and some users may want to skip it.

Exporting iterations can be quite handy when debugging or trying to make sense of why your model doesn't converge. Moreover, even when everything works as a dream, you might want to visualize how convergence is reached, for instance to distinguish between global and local effects. 

We stress that not only which variables to export but also when you wish to export them may vary between applications. In the model provided below, we export at all iterations using a separate exporter, keeping track of time step and iteration number using the vtu file suffix and collecting them using a single pvd file. The "time step" suffix of an iteration file is the sum of the iteration index and the product of the current time step index and $r$, which is the smallest power of ten exceeding the maximum number of non-linear iterations. 

Expecting that the simulation may crash or be stopped at any point during, we (over)write a pvd file each time a new vtu file is added. Alternative approaches and refinements include writing one pvd file for each time step and writing debugging files on some condition, e.g. that the iteration index exceeds some threshold.

In [4]:
class IterationExporting(TailoredExporting):
    def prepare_simulation(self):
        """Initialize iteration exporter.
    
        """
        super().prepare_simulation()
        # Setting export_constants_separately to False facilitates operations such as 
        # filtering by dimension in ParaView and is done here for illustrative purposes.
        self.iteration_exporter = pp.Exporter(
            self.mdg,
            file_name=self.params["file_name"]+"_iterations",
            folder_name=self.params["folder_name"],
            export_constants_separately=False, 
        )
        # Export initial solution for good measure
        self._export_iteration()
        
    def _iteration_export_data(self):
        """Returns data for iteration exporting.
        
        All data must be collected outside the exporter, since the values
        are updated in the pp.ITERATE subdictionary, not in pp.STATE.
        
        Returns: 
            Any type compatible with data argument of pp.Exporter().write_vtu().
        """
        # Retrieve values for displacement in the matrix...
        matrix = self._nd_subdomain()
        u = self.mdg.subdomain_data(matrix)[pp.STATE][pp.ITERATE][self.displacement_variable]
        displacements = [(matrix, self.displacement_variable, u)]
        
        # ... displacement jumps on fractures...
        jumps = list()
        fractures = self.mdg.subdomains(dim=self.nd-1)
        vals =  self._displacement_jump(fractures).evaluate(self.dof_manager).val
        projection = pp.ad.SubdomainProjections(
            subdomains=fractures, nd=self.nd
        )
        for sd in fractures:
            restriction = projection.cell_restriction([sd]).parse(self.mdg)
            jumps.append((sd, "displacement_jump", restriction * vals))
        
        # ... and pressures in all subdomains.
        pressures = list()
        for sd, sd_data in self.mdg.subdomains(return_data=True):
            vals = sd_data[pp.STATE][pp.ITERATE][self.scalar_variable]
            pressures.append((sd, "pressure", vals))
        
        # Collect and return all fields.
        data =  displacements + jumps + pressures
        return data
    
    def _export_iteration(self):
        """Export current solution to vtu files.
        
        This method is typically called by _after_newton_iteration.
        
        Having a separate exporter for iterations avoids distinguishing between iterations
        and time steps in the regular exporter's history (used for export_pvd).
        """
        # To make sure the nonlinear iteration index does not interfere with the 
        # time part, we multiply the latter by the next power of ten above the
        # maximum number of nonlinear iterations. Default value set to 10 in 
        # accordance with the default value used in NewtonSolver
        n = self.params.get("max_iterations", 10)
        p = round(np.log10(n))
        r = 10 ** p
        if r <= n:
            r = 10 ** (p+1) 
        self.iteration_exporter.write_vtu(self._iteration_export_data(), 
                                          time_dependent=True, 
                                          time_step=self._nonlinear_iteration + r * self.time_index)
        
    def after_newton_iteration(self, solution_vector: np.ndarray) -> None:
        """Add iteration export.
        
        Order of operations is important, super call distributes the solution to 
        iterate subdictionary.
        """
        super().after_newton_iteration(solution_vector)
        self._export_iteration()
        self.iteration_exporter.write_pvd()
        
        
class IterationCombined(IterationExporting, MDPoroelasticity):
    pass
    
params.update({
    "end_time": 2,
    "file_name": "debug",
    }
)
model_3 = IterationCombined(params)
pp.run_time_dependent_model(model_3, params)